In [1]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Rescaling, GlobalAveragePooling2D, Input, Flatten, RandomBrightness, RandomContrast, Dropout, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping
from huggingface_hub import HfApi
from datetime import datetime
from dotenv import load_dotenv
import os

c:\Users\donof\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
api = HfApi()

In [4]:
IMG_SIZE = 180

In [5]:
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)


In [6]:
train_set = datagen_train.flow_from_directory("../dataset/ttte/train", batch_size=8, target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

Found 200 images belonging to 8 classes.


In [7]:
test_set = datagen_test.flow_from_directory("../dataset/ttte/validation", batch_size=8, target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

Found 40 images belonging to 8 classes.


In [8]:
train_set.class_indices

{'Edward': 0,
 'Emily': 1,
 'Gordon': 2,
 'Henry': 3,
 'James': 4,
 'Percy': 5,
 'Thomas': 6,
 'Toby': 7}

In [9]:
test_set.class_indices

{'Edward': 0,
 'Emily': 1,
 'Gordon': 2,
 'Henry': 3,
 'James': 4,
 'Percy': 5,
 'Thomas': 6,
 'Toby': 7}

In [10]:
STEP_SIZE_TRAIN = train_set.samples // train_set.batch_size
STEP_SIZE_TRAIN

25

In [11]:
STEP_SIZE_TEST = test_set.samples // test_set.batch_size
STEP_SIZE_TEST

5

In [12]:
data_augmentation = Sequential(
    [

    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1)
    ]
)
data_augmentation

<Sequential name=sequential, built=False>

In [13]:
input_shape = IMG_SIZE, IMG_SIZE, 3
base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False,  
    weights='imagenet'
)
base_model.trainable = True

base_model.summary()

C:\Users\donof\AppData\Local\Temp\ipykernel_8440\4258986162.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


Model: "mobilenetv2_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 180, 180,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 90, 90,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 90, 90,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 90, 90,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 90, 90,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 90, 90,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 90, 90,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 90, 90,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 90, 90,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 90, 90,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 90, 90,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 90, 90,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 91, 91,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 45, 45,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 45, 45,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 45, 45,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 45, 45,    │      2,304 │ block_1_depthwis

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 2,223,872 (8.48 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [14]:
for layer in base_model.layers[:-50]:
    layer.trainable = False

In [15]:
model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    data_augmentation,
    # Rescaling(1./255),
    base_model,
    Flatten(),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(train_set.class_indices), activation='softmax')
    # Conv2D(16,(3,3), padding='same', activation = 'relu'),
    # MaxPooling2D(padding='same'),
    # Conv2D(32,(3,3), padding = 'same', activation = 'relu'),
    # MaxPooling2D(padding='same'),
    # Conv2D(64,(3,3), padding = 'same', activation = 'relu'),
    # MaxPooling2D(padding='same'),
    # Dropout(0.3),
    # Flatten(),
    # Dense(256, activation = 'relu'),
    # Dropout(0.4),
    # Dense(len(train_set.class_indices), activation='softmax')
])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 6, 6, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 46080)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 46080)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    11,796,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,056,776 (53.62 MB)

 Trainable params: 13,653,896 (52.09 MB)

 Non-trainable params: 402,880 (1.54 MB)

In [16]:
# for layer in base_model.layers[:-30]:
#     layer.trainable = False

In [17]:
opt = Adam(learning_rate=1e-4)

callback = EarlyStopping(monitor='val_accuracy',
                         patience=5,
                         verbose=1,
                         mode="max",
                         baseline=0.5,
                          restore_best_weights=True,
                          start_from_epoch=5
                         )


In [18]:

model.compile(
            optimizer=opt,
              loss = CategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

history = model.fit(train_set,
                           epochs=15,
                           steps_per_epoch=STEP_SIZE_TRAIN,
                           validation_data=test_set, 
                            validation_steps=STEP_SIZE_TEST,
                           verbose=1,
                           callbacks=[callback]
                           )


Epoch 1/15


c:\Users\donof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\backend\tensorflow\nn.py:675: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 361ms/step - accuracy: 0.2800 - loss: 3.1604 - val_accuracy: 0.4000 - val_loss: 2.0124
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 275ms/step - accuracy: 0.4850 - loss: 1.7015 - val_accuracy: 0.4250 - val_loss: 1.8218
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 273ms/step - accuracy: 0.6400 - loss: 1.0217 - val_accuracy: 0.6000 - val_loss: 1.1147
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 267ms/step - accuracy: 0.7550 - loss: 0.6871 - val_accuracy: 0.6500 - val_loss: 1.4587
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 272ms/step - accuracy: 0.7900 - loss: 0.5642 - val_accuracy: 0.6000 - val_loss: 1.3052
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 288ms/step - accuracy: 0.8650 - loss: 0.4255 - val_accuracy: 0.7000 - val_loss: 1.2498
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 292ms/step - accuracy: 0.8900 - loss: 0.3123 - val_accuracy: 0.6250 - val_loss: 1.5704
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 281ms/step - accuracy: 0.8650 - loss: 0.4125 - val_accuracy: 0.7750 - val

In [19]:

converter = tensorflow.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\donof\AppData\Local\Temp\tmpweu02puo\assets


INFO:tensorflow:Assets written to: C:\Users\donof\AppData\Local\Temp\tmpweu02puo\assets


Saved artifact at 'C:\Users\donof\AppData\Local\Temp\tmpweu02puo'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2106195376992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2106195376288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2105525344160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2106195376464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2106195740272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2106195746256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2106195745200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2106195747664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2106195743440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2106195746432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2

In [20]:
with open("ttte_classifier.tflite", "wb") as f:
    f.write(tflite_model)

In [21]:
api.upload_file(
    path_or_fileobj="./ttte_classifier.tflite",
    path_in_repo="ttte_classifier.tflite",
    repo_id=os.getenv("REPO"),
    revision="main",
    create_pr=True,
    repo_type="model"
)

ttte_classifier.tflite: 100%|██████████| 56.1M/56.1M [00:02<00:00, 26.8MB/s]


CommitInfo(commit_url='https://huggingface.co/donoftime2018/TTTE-Model/commit/c6e541e412773f25631498b41e8b03e8e4f5f651', commit_message='Upload ttte_classifier.tflite with huggingface_hub', commit_description='', oid='c6e541e412773f25631498b41e8b03e8e4f5f651', pr_url='https://huggingface.co/donoftime2018/TTTE-Model/discussions/3', repo_url=RepoUrl('https://huggingface.co/donoftime2018/TTTE-Model', endpoint='https://huggingface.co', repo_type='model', repo_id='donoftime2018/TTTE-Model'), pr_revision='refs/pr/3', pr_num=3)